In [33]:
import tensorflow as tf
import keras_losses as kl
import numpy as np
import metrics as mets
import keras_tools as kt

In [34]:
#MPE

In [64]:
# data
x_train = np.genfromtxt("../../data/21_cm/8_params_21_2_x_tr.csv", delimiter=",")
y_train = np.genfromtxt("../../data/21_cm/8_params_21_2_y_tr.csv", delimiter=",")
y_train = y_train.reshape(-1,1) #keras outputs y_pred as 2d, so make y_true 2d as well

In [65]:
#architecture parameters
num_inputs = x_train.shape[1] #aka n in ml terminology
num_outputs = y_train.shape[1] #dimensionality of output y (for single record)
layer_sizes = [32, 64, 64, 32]
#propagation parameters
epochs = 100
m = x_train.shape[0] #total number of records
batch_num = m / 2

In [66]:
Model = tf.keras.models.Model
Input = tf.keras.layers.Input
Dense = tf.keras.layers.Dense
Dropout = tf.keras.layers.Dropout
Batch_norm = tf.keras.layers.BatchNormalization

In [67]:
#mlp networks
#create model
def mlp_drop_norm(num_inputs, num_outputs, layer_sizes):
    """
    arbitrary sized mlp with tanh activation, dropout, batch norm
    """
    dropout_reg = 0.00001
    a0 = Input(shape = (num_inputs,))
    inputs = a0
    for layer_size in layer_sizes:
        outputs = Dense(layer_size, activation = 'tanh')(a0)
        outputs = Dropout(dropout_reg)(outputs)
        outputs = Batch_norm()(outputs)
        a0 = outputs
    #don't want dropout and normalisation for output layer. linear activation
    outputs = Dense(num_outputs, activation = 'linear')(a0)
    model = Model(inputs = inputs, outputs = outputs)
    return model

def mlp_norm(num_inputs, num_outputs, layer_sizes):
    """
    arbitrary sized mlp with tanh activation, batch norm
    """
    a0 = Input(shape = (num_inputs,))
    inputs = a0
    for layer_size in layer_sizes:
        outputs = Dense(layer_size, activation = 'tanh')(a0)
        outputs = Batch_norm()(outputs)
        a0 = outputs
    #don't want normalisation for output layer. linear activation
    outputs = Dense(num_outputs, activation = 'linear')(a0)
    model = Model(inputs = inputs, outputs = outputs)
    return model

def mlp(num_inputs, num_outputs, layer_sizes):
    """
    arbitrary sized mlp with tanh activation
    """
    a0 = Input(shape = (num_inputs,))
    inputs = a0
    for layer_size in layer_sizes:
        outputs = Dense(layer_size, activation = 'tanh')(a0)
        a0 = outputs
    #linear activation for output layer
    outputs = Dense(num_outputs, activation = 'linear')(a0)
    model = Model(inputs = inputs, outputs = outputs)
    return model


In [68]:
#ResNet networks
def uap_mlp_ResNet_block(num_inputs, a0):
    """
    copied from keras_models.py
    """
    a1 = Dense(1, activation = 'relu')(a0)
    a2_part = Dense(num_inputs, activation = 'linear', use_bias = False)(a1)
    return tf.keras.layers.Add()([a0, a2_part])

def coursera_mlp_ResNet_block(num_inputs, a0):
    """
    copied from keras_models.py
    """
    a1 = Dense(1, activation = 'relu')(a0)
    z2_part = Dense(num_inputs, activation = 'linear')(a1)
    z2 = tf.keras.layers.Add()([a0, z2_part])
    return tf.keras.layers.Activation('relu')(z2)

def same_mlp_ResNet_block(num_inputs, a0):
    """
    copied from keras_models.py
    """
    a1 = Dense(num_inputs, activation = 'relu')(a0)
    z2_part = Dense(num_inputs, activation = 'linear')(a1)
    z2 = tf.keras.layers.Add()([a0, z2_part])
    return tf.keras.layers.Activation('relu')(z2)

def mlp_ResNet_1(num_inputs, num_outputs, layer_sizes, ResNet_type = 'uap'):
    """
    all layers are size of input of nn, apart from final layer which obvs conforms to output
    """
    num_blocks = 4
    if ResNet_type == 'uap':
        ResNet_block = uap_mlp_ResNet_block
    elif ResNet_type == 'coursera':
        ResNet_block = coursera_mlp_ResNet_block
    elif ResNet_type == 'same':
        ResNet_block = same_mlp_ResNet_block
    else:
        raise NotImplementedError
    a0 = Input(shape = (num_inputs,))
    inputs = a0
    for _ in range(num_blocks):
        a2 = ResNet_block(num_inputs, a0)
        a0 = a2
    outputs = Dense(num_outputs, activation = 'linear')(a0)
    return Model(inputs = inputs, outputs = outputs)

In [69]:
model = mlp(num_inputs, num_outputs, layer_sizes)

In [70]:
mean = -50.35104929 
var = 5410.76986054
n_z = 136
twenty_one_err_met = kl.twenty_one_cm_rmse_higher_order(mean, var)
twenty_one_err_met_ts = kl.twenty_one_cm_rmse_higher_order_ts_mean(mean, var, n_z, m) #won't work, too computationally expensive

In [71]:
adam = tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.summary()
model.compile(optimizer=adam,
              loss=['mean_squared_error'],
              metrics=[twenty_one_err_met])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 8)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 32)                288       
_________________________________________________________________
dense_21 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_22 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_23 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 33        
Total params: 8,673
Trainable params: 8,673
Non-trainable params: 0
_________________________________________________________________


In [72]:
initial_epoch = 0

In [73]:
#train model
history = model.fit(x_train, y_train,
                    batch_size=batch_num,
                    epochs=epochs, initial_epoch = initial_epoch,
                    verbose=1)
initial_epoch += epochs
epochs += epochs

#save model
#model.save("../../MPE_examples/saved_keras_models/21/8_params_21_mlp_1.h5")

Epoch 1/100
2932432/2932432 [==============================] - 5s 2us/step - loss: 1.4487 - twenty_one_cm_rmse: 0.2086
Epoch 2/100
2932432/2932432 [==============================] - 4s 1us/step - loss: 1.0535 - twenty_one_cm_rmse: 0.1857
Epoch 3/100
2932432/2932432 [==============================] - 4s 1us/step - loss: 1.0270 - twenty_one_cm_rmse: 0.1658
Epoch 4/100
2932432/2932432 [==============================] - 4s 2us/step - loss: 0.9759 - twenty_one_cm_rmse: 0.1617
Epoch 5/100
2932432/2932432 [==============================] - 4s 1us/step - loss: 0.9283 - twenty_one_cm_rmse: 0.1725
Epoch 6/100
2932432/2932432 [==============================] - 4s 1us/step - loss: 0.9012 - twenty_one_cm_rmse: 0.1554
Epoch 7/100
2932432/2932432 [==============================] - 4s 1us/step - loss: 0.8542 - twenty_one_cm_rmse: 0.1513
Epoch 8/100
1466216/2932432 [==============>...............] - ETA: 2s - loss: 0.8292 - twenty_one_cm_rmse: 0.1939

KeyboardInterrupt: 

In [74]:
print y_train.mean()
y_pred = model.predict(x_train)
print y_pred.mean()
print y_train.var()
print y_pred.var()
print 'rmse ts mean'
print mets.twenty_one_cm_rmse_ts_mean_higher_order(y_train, y_pred, mean, var, n_z)

6.637216691790522e-17
(2932432,)
-0.027518231
(2932432, 1)
0.9999999999999993
0.11563098
rmse ts mean


In [ ]:
model.get_weights()

In [ ]:
print kt.get_activations(x_train, model, 0)

In [49]:
print kt.get_gradients(x_train, y_train, model, 0)

6.637216691790522e-17
0.6600129
0.9999999999999993
0.10705553
rmse ts mean
-50.35104929000002
-1.8018477
5410.769860539998
579.2528
0.3423576396435991
